In [1]:
%pip install brainflow
%pip install matplotlib
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import numpy as np
import matplotlib.pyplot as plt
import time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!git clone https://github.com/slSeanWU/Compose_and_Embellish
!git lfs pull
%pip install -r requirements.txt
%pip install git+https://github.com/cifkao/fast-transformers.git@39e726864d1a279c9719d33a95868a4ea2fb5ac5
!git clone https://huggingface.co/slseanwu/compose-and-embellish-pop1k7

Cloning into 'Compose_and_Embellish'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 110 (delta 28), reused 27 (delta 16), pack-reused 62 (from 1)
Receiving objects: 100% (110/110), 72.49 KiB | 1.34 MiB/s, done.
Resolving deltas: 100% (45/45), done.
[Errno 2] No such file or directory: '/Users/noorelbanna/Desktop/ProjectMUSEEG/ProjectB/Compose_and_Embellish'
/Users/noorelbanna/Desktop/ProjectMUSEEG/ProjectB/compositions
git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	refs
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/cifkao/fast-transformers.git (to revision 39e726864d1a279c9719d33a95868a4ea2fb5ac5) to /private/var/folders/nr/w1b31n2n1956hqwqf59y6kw40000gn/T/pip-req-build-6f6qv06w
  Running command git clone

# Setting up BrainFlow

## Connecting with board

In [3]:
PARAMS = BrainFlowInputParams()

In [4]:
try:
    PARAMS.serial_port = "/dev/cu.usbserial-DP04VZSH"
    board_ID = BoardIds.CYTON_DAISY_BOARD
    board = BoardShim(board_ID, PARAMS)
    board.prepare_session()
    print("Successfully connected to Cyton+Daisy board.")
except Exception as e:
    print(f"Error connecting to Cyton+Daisy board: {e}")
    input("Want to use synthetic board instead? Press Enter to continue...")
    board_ID = BoardIds.SYNTHETIC_BOARD
    board = BoardShim(board_ID, PARAMS)
    board.prepare_session()
    print("Successfully connected to synthetic board.")

board.release_session()

# testing sampling rate and data acquisition
print("starting stream...")
board.prepare_session()
board.start_stream()
time.sleep(5)  # collect data for 5 seconds
data = board.get_board_data()
print("stopping stream...")
board.stop_stream()
board.release_session()

[2025-10-28 14:43:52.103] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DP04VZSH",
    "timeout": 0
}


Error connecting to Cyton+Daisy board: UNABLE_TO_OPEN_PORT_ERROR:2 unable to prepare streaming session


[2025-10-28 14:43:52.103] [board_logger] [info] opening port /dev/cu.usbserial-DP04VZSH
[2025-10-28 14:43:52.103] [board_logger] [error] Make sure you provided correct port name and have permissions to open it(run with sudo/admin). Also, close all other apps using this port.
[2025-10-28 14:43:54.652] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DP04VZSH",
    "timeout": 0
}
[2025-10-28 14:43:54.656] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
  

Successfully connected to synthetic board.
starting stream...
stopping stream...


In [5]:
print(type(data))
print(data.shape)  # should be (num_channels, num_samples)

<class 'numpy.ndarray'>
(32, 1251)


# Model Finetuning + Implementation


In [22]:
import sys
sys.path.append("/Users/noorelbanna/Desktop/ProjectMUSEEG/Compose_and_Embellish/")

import torch
import yaml
from stage01_compose.inference_utils import generate_plain_xl

sys.argv = [
    'inference.py',  # script name
    '/Users/noorelbanna/Desktop/ProjectMUSEEG/Compose_and_Embellish/stage01_compose/config/pop1k7_finetune.yaml',  # config file
    'generation/stage01',  # output folder
    '20'  # number of pieces
]
from stage01_compose.inference import read_vocab, dump_midi, PlainTransformer, TempoEvent
#from stage02_embellish.inference_gpt2 import embellish_from_leadsheet
import mido

[nucleus parameters] t = 1.2, p = 0.97


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
def compose_from_notes(notes, config_path="stage01_compose/config/pop1k7_finetune.yaml",
                       output_path="generation/stage01/sample.mid",
                       temp=1.2, top_p=0.97, max_dec_len=2400, max_bars=128):

    # Load config
    config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)
    event2idx, idx2event, vocab_size = read_vocab(config['data']['vocab_path'])

    # Load model
    mconf = config['model']
    model = PlainTransformer(
        mconf['d_word_embed'], vocab_size,
        mconf['decoder']['n_layer'], mconf['decoder']['n_head'],
        mconf['decoder']['d_model'], mconf['decoder']['d_ff'],
        mconf['decoder']['tgt_len'], mconf['decoder']['tgt_len'],
        dec_dropout=mconf['decoder']['dropout'],
        pre_lnorm=mconf['pre_lnorm']
    ).cuda()
    pretrained_dict = torch.load(config['inference_param_path'], map_location='cpu')
    model.load_state_dict(pretrained_dict)
    model.eval()

    # Convert notes to primer format
    primer = notes + ["Bar_None"]

    # Generate lead sheet
    gen_words, t_sec = generate_plain_xl(
        model, event2idx, idx2event,
        max_events=max_dec_len, max_bars=max_bars,
        primer=primer, temp=temp, top_p=top_p
    )

    # Save as MIDI
    dump_midi(gen_words, idx2event, output_path)
    return output_path


## Isolate EEG channels

In [ ]:
EEG_CHANNELS = BoardShim.get_eeg_channels(board_ID)
print(f"EEG channels: {EEG_CHANNELS}")
eeg_data = data[EEG_CHANNELS]
print(f"EEG data shape: {eeg_data.shape}")  # should be (num_eeg_channels, num_samples)

## Note Mapping

In [ ]:
NOTE_RANGE = (0, 127)  # Full MIDI note range (C1 to G9)

# === OPEN MIDI OUTPUT ===
print("Available MIDI outputs:")
print(mido.get_output_names())
outport = mido.open_output('GarageBand Virtual In')
print("connected to MIDI output: GarageBand Virtual In")

# === CONNECT TO BRAINFLOW STREAM ===
print("starting stream...")
board.prepare_session()
board.start_stream()

# get 10 second of data to process
board.insert_marker("starting base of notes")
time.sleep(10)  # collect data for 10 seconds
board.insert_marker("ending of base notes")
starting_data = board.get_board_data()

# Helper: map EEG voltage to MIDI note
def voltage_to_note(v, vmin=-100, vmax=100):
    v = np.clip(v, vmin, vmax)
    return int(np.interp(v, [vmin, vmax], NOTE_RANGE))

In [ ]:
# markers
MARKER_CHANNEL = BoardShim.get_marker_channel(board_ID)
print(f"Marker channel: {MARKER_CHANNEL}")
marker_data = data[MARKER_CHANNEL]
print(f"Marker data shape: {marker_data.shape}")
print("Marker data: ", marker_data)  # print marker data to see if there are any markers, there should be none

In [ ]:
while True:
    try:
        # Get a sample from LSL
        current_data = board_shim.get_current_board_data(10)

        # Map EEG value to MIDI note
        note = voltage_to_note(current_data[0][0])  # assuming channel 0

        # Send MIDI note on & off
        msg_on = mido.Message('note_on', note=note, velocity=64)
        msg_off = mido.Message('note_off', note=note)

        outport.send(msg_on)
        print(f"🎵 Playing note: {note} (EEG value: {eeg_value:.2f})")
        time.sleep(0.1)
        outport.send(msg_off)

    except KeyboardInterrupt:
        print("\n Stopped by user.")
        break


In [ ]:
context_notes = []
context_tokens = None

while True:
    # 1️⃣ Collect your new notes every 15 seconds
    new_notes = get_new_notes()  # Your own function, returns list of (pitch, dur, time)
    context_notes.extend(new_notes)

    # Optional: keep only recent context to avoid drift
    context_notes = context_notes[-MAX_NOTES:]

    # 2️⃣ Compose: Generate lead sheet (melody + chords)
    leadsheet_tokens = compose_from_notes(context_notes)

    # 3️⃣ Embellish: Generate full piano performance
    embellished_tokens = embellish_from_leadsheet(leadsheet_tokens, prev_context=context_tokens)

    # 4️⃣ Save to MIDI
    save_midi(embellished_tokens, "live_output.mid")

    # Update rolling context for smooth transitions
    context_tokens = embellished_tokens[-CONTEXT_LEN:]

    # 5️⃣ Wait for next input batch
    time.sleep(15)